# 深度学习进行词性标注（使用RNN和BRNN模型）

## 1. 导入数据集

In [1]:
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
import gensim

# 训练集和测试集的地址
train_data = './data/ctb5.1-pos/train.tsv'
test_data = './data/ctb5.1-pos/test.tsv'

## 2. 数据读取函数

In [2]:
# 读取数据集，并提取出其中的文本部分和标签部分
def get_data(file_path):
    data = pd.read_csv(file_path, sep='\t', 
                       skip_blank_lines=False, 
                       header=None,
                      )
    
    # 取出文本部分
    content = data[0]
    
    # 取出标签部分
    label = data[1]
    
    return content, label

In [3]:
# 使用get_data函数获取训练集和测试集的文本和标签
X_train, y_train = get_data(train_data)
X_test, y_test = get_data(test_data)

## 3. 数据预处理与格式转化

In [4]:
## 构建标签字典
# 合并训练集和测试集的标签
labels = y_train.tolist() + y_test.tolist()
# 利用集合进行去重，再转化为列表格式，获得标签类型列表
labels_types = list(set(labels))
print("训练集和测试集的总标签类别数len(labels_types)为：" + str(len(labels_types)))
print("标签类型列表为：\n", labels_types)

# 构建标签频次字典（标签：该标签出现次数）
labels_dict = {}

# 构建标签索引字典（标签： 该标签的索引）
# 加入{"padded_label" : 0}降低损失率
labels_index = {"padded_label" : 0}

for index in range(len(labels_types)):
    # 根据标签类别列表通过索引获取标签
    label = labels_types[index]
    # 更新标签频次字典
    labels_dict.update({label: labels.count(label)}) 
    # 更新标签索引字典
    labels_index.update({label: index+1}) 
    
print("\n训练集和测试集的总标签数len(labels)为：" + str(len(labels)))

# 输出构建好的标签频次字典
print("\n标签字典的对象数len(labels_dict)为：" + str(len(labels_dict)))
print("标签频次字典内容labels_dict为：\n", labels_dict)

# 输出构建好的标签索引字典
print("\n标签索引字典内容labels_dict为：\n", labels_index)

训练集和测试集的总标签类别数len(labels_types)为：36
标签类型列表为：
 [nan, 'PU', 'NN', 'CC', 'NP', 'NT', 'DEG', 'CD', 'DT', 'LB', 'BA', 'IJ', 'JJ', 'MSP', 'VP', 'FW', 'PN', 'VA', 'CS', 'NR', 'X', 'DEC', 'M', 'AD', 'DER', 'VE', 'AS', 'VV', 'VC', 'DEV', 'P', 'SB', 'OD', 'LC', 'ETC', 'SP']

训练集和测试集的总标签数len(labels)为：520125

标签字典的对象数len(labels_dict)为：36
标签频次字典内容labels_dict为：
 {nan: 18426, 'PU': 76753, 'NN': 136643, 'CC': 7355, 'NP': 5, 'NT': 9659, 'DEG': 12337, 'CD': 16182, 'DT': 5986, 'LB': 245, 'BA': 755, 'IJ': 12, 'JJ': 13234, 'MSP': 1336, 'VP': 1, 'FW': 33, 'PN': 6644, 'VA': 7755, 'CS': 892, 'NR': 30570, 'X': 6, 'DEC': 12510, 'M': 13790, 'AD': 36430, 'DER': 258, 'VE': 3005, 'AS': 4118, 'VV': 69858, 'VC': 5404, 'DEV': 634, 'P': 17606, 'SB': 455, 'OD': 1675, 'LC': 7782, 'ETC': 1303, 'SP': 468}

标签索引字典内容labels_dict为：
 {'padded_label': 0, nan: 1, 'PU': 2, 'NN': 3, 'CC': 4, 'NP': 5, 'NT': 6, 'DEG': 7, 'CD': 8, 'DT': 9, 'LB': 10, 'BA': 11, 'IJ': 12, 'JJ': 13, 'MSP': 14, 'VP': 15, 'FW': 16, 'PN': 17, 'VA': 18, 'CS':

In [5]:
# 按句子对X、y进行拆分
def split_corpus_by_sentence(content):
    cleaned_sentence = []
    split_label = content.isnull() # 用来判断是否有缺失值，返回布尔值
    last_split_index = 0
    index = 0
    
    while index < len(content):
        current_word = content[index]
        # 如果有缺失值且cleaned_sentence列表中无内容
        if split_label[index] == True and len(cleaned_sentence) == 0:
            # 添加content列表中从索引last_split_index到index的内容到cleaned_sentence
            cleaned_sentence.append(np.array(content[last_split_index: index]))
            last_split_index = index + 1
            index += 1
            
        # 如果有缺失值且cleaned_sentence列表中有内容
        elif split_label[index] == True and len(cleaned_sentence) > 0:
            cleaned_sentence.append(np.array(content[last_split_index: index]))
            last_split_index = index + 1
            index += 1
            
        else:
            index += 1
    return cleaned_sentence

# 利用上述函数按句子拆分训练集和测试集的文本和标签
X_train_sent_split = split_corpus_by_sentence(X_train)
y_train_sent_split = split_corpus_by_sentence(y_train)
X_test_sent_split = split_corpus_by_sentence(X_test)
y_test_sent_split = split_corpus_by_sentence(y_test)

print('训练集中按句进行拆分后的句子为：\n', X_train_sent_split[:5])
print('\n训练集中按句进行拆分后的句子所对应的词性为：\n', y_train_sent_split[:5])

训练集中按句进行拆分后的句子为：
 [array(['上海', '浦东', '开发', '与', '法制', '建设', '同步'], dtype=object), array(['新华社', '上海', '二月', '十日', '电', '（', '记者', '谢金虎', '、', '张持坚', '）'],
      dtype=object), array(['上海', '浦东', '近年', '来', '颁布', '实行', '了', '涉及', '经济', '、', '贸易', '、',
       '建设', '、', '规划', '、', '科技', '、', '文教', '等', '领域', '的', '七十一', '件',
       '法规性', '文件', '，', '确保', '了', '浦东', '开发', '的', '有序', '进行', '。'],
      dtype=object), array(['浦东', '开发', '开放', '是', '一', '项', '振兴', '上海', '，', '建设', '现代化',
       '经济', '、', '贸易', '、', '金融', '中心', '的', '跨世纪', '工程', '，', '因此',
       '大量', '出现', '的', '是', '以前', '不', '曾', '遇到', '过', '的', '新', '情况',
       '、', '新', '问题', '。'], dtype=object), array(['对', '此', '，', '浦东', '不', '是', '简单', '的', '采取', '“', '干', '一', '段',
       '时间', '，', '等', '积累', '了', '经验', '以后', '再', '制定', '法规', '条例', '”',
       '的', '做法', '，', '而', '是', '借鉴', '发达', '国家', '和', '深圳', '等', '特区',
       '的', '经验', '教训', '，', '聘请', '国内外', '有关', '专家', '学者', '，', '积极',
       '、', '及时', '地', '制定', '和',

In [6]:
# 根据之前构建的标签字典将标签文本转换为索引
def transfer_label_category_index(origin_labels, labels_types):
    transfered_label = []
    for sentence_labels in origin_labels:
        # 将标签依据字典转换为序号
        labels_format_index = [labels_types.index(label) for label in sentence_labels]
        transfered_label.append(labels_format_index)
    return transfered_label

# 利用上述函数将训练集和测试集的标签转换为索引
y_train_index = transfer_label_category_index(y_train_sent_split, labels_types)
y_test_index = transfer_label_category_index(y_test_sent_split, labels_types)

print('训练集中的标签转换为索引对应为：\n', y_train_index[:5])
print('\n测试集中的标签转换为索引对应为：\n', y_test_index[:5])

训练集中的标签转换为索引对应为：
 [[19, 19, 2, 3, 2, 2, 27], [2, 19, 5, 5, 2, 1, 2, 19, 1, 19, 1], [19, 19, 5, 33, 27, 27, 26, 27, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 34, 2, 21, 7, 22, 2, 2, 1, 27, 26, 19, 2, 6, 12, 2, 1], [19, 2, 2, 28, 7, 22, 27, 19, 1, 27, 2, 2, 1, 2, 1, 2, 2, 21, 12, 2, 1, 23, 23, 27, 21, 28, 5, 23, 23, 27, 26, 21, 12, 2, 1, 12, 2, 1], [30, 16, 1, 19, 23, 28, 17, 29, 27, 1, 27, 7, 22, 2, 1, 30, 27, 26, 2, 33, 23, 27, 2, 2, 1, 21, 2, 1, 3, 28, 27, 12, 2, 3, 19, 34, 2, 6, 2, 2, 1, 27, 2, 12, 2, 2, 1, 23, 1, 23, 29, 27, 3, 27, 2, 2, 1, 27, 8, 2, 2, 23, 27, 23, 31, 27, 2, 2, 1]]

测试集中的标签转换为索引对应为：
 [[19, 19, 30, 19, 27, 12, 2, 2, 2], [19, 19, 5, 5, 2, 1, 2, 19, 1, 19, 1], [1, 19, 19, 2, 12, 2, 2, 2, 2, 1, 5, 30, 19, 27, 1], [5, 30, 16, 27, 21, 28, 2, 2, 2, 2, 2, 3, 2, 2, 2, 7, 22, 2, 1, 23, 23, 27, 26, 2, 2, 2, 2, 2, 1], [8, 7, 22, 2, 28, 23, 30, 2, 2, 2, 2, 2, 2, 2, 2, 19, 2, 3, 19, 19, 12, 2, 2, 1, 30, 19, 2, 2, 2, 2, 1, 2, 2, 2, 1, 19, 2, 27, 21, 1]]


In [7]:
# 设置句子的最大长度为100
MAX_SEQUENCE_LENGTH = 100

## 进行标签格式转化
# 利用zeros(shape, dtype=float, order='C')构建张量，值全为0
# 形状shape对应（标签样本数，句子长度，标签类别数）
y_train_index_padded = np.zeros((len(y_train_index), MAX_SEQUENCE_LENGTH, len(labels_types)+1), # 形状 
                                dtype='float', # 数据类型 
                                order='C',     # c代表与c语言类似，行优先
                               )
y_test_index_padded = np.zeros((len(y_test_index), MAX_SEQUENCE_LENGTH, len(labels_types)+1), # 形状 
                              dtype='float', # 数据类型 
                              order='C',     # c代表与c语言类似，行优先
                              )

''' 
填充张量
嵌套循环遍历：先句子后词
'''
# 训练集
for sentence_labels_index in range(len(y_train_index)):
    for label_index in range(len(y_train_index[sentence_labels_index])):
        if label_index < MAX_SEQUENCE_LENGTH:
            y_train_index_padded[sentence_labels_index, label_index, y_train_index[sentence_labels_index][label_index]+1] = 1
    # 优化：若为填充的标签，则将其预测为第一位为1        
    if len(y_train_index[sentence_labels_index]) < MAX_SEQUENCE_LENGTH:
        for label_index in range(len(y_train_index[sentence_labels_index]), MAX_SEQUENCE_LENGTH):
            y_train_index_padded[sentence_labels_index, label_index, 0] = 1

# 测试集      
for sentence_labels_index in range(len(y_test_index)):
    for label_index in range(len(y_test_index[sentence_labels_index])):
        if label_index < MAX_SEQUENCE_LENGTH:
            y_test_index_padded[sentence_labels_index, label_index, y_test_index[sentence_labels_index][label_index]+1] = 1        
    # 优化：若为填充的标签，则将其预测为第一位为1 
    if len(y_test_index[sentence_labels_index]) < MAX_SEQUENCE_LENGTH:
        for label_index in range(len(y_test_index[sentence_labels_index]), MAX_SEQUENCE_LENGTH):
            y_test_index_padded[sentence_labels_index, label_index, 0] = 1
    
print('训练集中填充的张量为：\n', y_train_index_padded[:1])
print('\n测试集中填充的张量为：\n', y_test_index_padded[:1])

训练集中填充的张量为：
 [[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]]

测试集中填充的张量为：
 [[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]]


## 4. word2vec模型导入
预训练的word2vec模型采用前人用中文维基百科训练好的模型

In [8]:
'''（1）导入 预训练的词向量'''

# 本地词向量的地址
myPath = './data/sgns.wiki.word'   

# 以二进制读取词向量
Word2VecModel = gensim.models.KeyedVectors.load_word2vec_format(myPath).wv

# 词语的向量，是numpy格式
vector = Word2VecModel.wv['空间']

D:\Anaconda3-5.3.0\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys
D:\Anaconda3-5.3.0\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # Remove the CWD from sys.path while we load stuff.


In [9]:
# 输出词向量的类型
print('词向量的类型为：', type(Word2VecModel.wv))  
# 结果为：Word2VecKeyedVectors

# 获取word2vec训练后model中的所有的词
for i, j in Word2VecModel.wv.vocab.items():
    # 此时 i 代表每个单词
    print(i) 
    
    # j 代表封装了 词频 等信息的 gensim“Vocab”对象
    # 例子：Vocab(count:1481, index:38, sample_int:3701260191)
    print(j)
    
    break

词向量的类型为： <class 'gensim.models.keyedvectors.Word2VecKeyedVectors'>
，
Vocab(count:352217, index:0)


D:\Anaconda3-5.3.0\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
D:\Anaconda3-5.3.0\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [10]:
'''（2）构造包含所有词语的list；
        初始化“词语-序号”字典和“词向量”矩阵
'''
# 存储所有的单词和gensim“Vocab”对象
vocab_list = [word for word, Vocab in Word2VecModel.wv.vocab.items()]

# 初始化 `[word : token]` ，后期 tokenize 语料库就是用该词典。
word_index = {" ": 0}

# 初始化`[word : vector]`字典
word_vector = {}

# 初始化 存储所有向量的大矩阵【其中多一位（首行）：词向量全为 0，用于 padding补零。
# 行数为：所有单词数+1。比如 10000+1 ； 
# 列数为：词向量“维度”。比如100。
embeddings_matrix = np.zeros((len(vocab_list)+1, Word2VecModel.vector_size))

D:\Anaconda3-5.3.0\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """


In [11]:
'''（3）填充上述的字典word_index和大矩阵embeddings_matrix'''

# 循环遍历 存储所有的单词和gensim“Vocab”对象的 vocab_list列表
for i in range(len(vocab_list)):
    #print(i)    
    # 将每个词语存储到vocab_list列表中
    word = vocab_list[i]    
    # 将每个词语及对应的序号存储到word_index字典中
    word_index[word] = i + 1   
    # 将每个{词语：词向量}存储到word_vector字典中
    word_vector[word] = Word2VecModel.wv[word]   
    # 将每个{词向量：序号+1}存储到词向量矩阵embeddings_matrix中
    embeddings_matrix[i + 1] = Word2VecModel.wv[word]
    
# np.save('x_word_index.npy', word_index)

D:\Anaconda3-5.3.0\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # This is added back by InteractiveShellApp.init_path()
D:\Anaconda3-5.3.0\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  del sys.path[0]


In [12]:
'''（4）序号化 文本；
        tokenizer句子；
        并返回每个句子所对应的词语索引
'''
from keras.preprocessing import sequence

# 由于将词语转化为索引的word_index需要与词向量模型对齐，故在导入词向量模型后再将X进行处理
def tokenizer(texts, word_index):
    data = []
    
    for sentence in texts:
        new_sentence = []
        for word in sentence:
            try:
                # 根据word_index字典把文本中的词语转化为index
                new_sentence.append(word_index[word])
            except:
                new_sentence.append(0)
        data.append(new_sentence)
        
    # 使用kears的内置函数padding对齐句子（好处是输出numpy数组，不用自己转化了）
    data = sequence.pad_sequences(data,
                                 maxlen = MAX_SEQUENCE_LENGTH, # 序列的最大长度（大于此长度的序列将被截短，小于此长度的序列将在后部填0）
                                 padding = 'post', # 当需要补0时，在序列的结尾补
                                 truncating = 'post', # 当需要截断序列时，从结尾截断
                                 )
    return data

# 将训练集中的词语转换为索引
X_train_tokenized = tokenizer(X_train_sent_split, word_index)

# 将测试集中的词语转换为索引
X_test_tokenized = tokenizer(X_test_sent_split, word_index)

print('训练集中的词语转换为索引为：\n', X_train_tokenized[:1])
print('\n测试集中的词语转换为索引为：\n', X_test_tokenized[:1])

训练集中的词语转换为索引为：
 [[  347 16980   507    10 15537   603  4380     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0]]

测试集中的词语转换为索引为：
 [[  28  523    6 5705 2208  287 1216  587 1523    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0   

## 5. 标引网络构建及训练评估

In [13]:
from keras.models import Sequential
from keras.layers import Embedding, Dense, Dropout, Bidirectional
from keras.layers.recurrent import SimpleRNN
import keras
from keras import optimizers

# 词向量维度为300
EMBEDDING_DIM = 300

### （1）RNN模型

In [14]:
# RNN模型
def RNN(embeddings_matrix, EMBEDDING_DIM, MAX_SEQUENCE_LENGTH, labels_types):
    model = Sequential()
    model.add(Embedding(input_dim = len(embeddings_matrix), # 字典长度
                        output_dim = EMBEDDING_DIM, # 词向量长度（300）
                        weights = [embeddings_matrix], # 预训练的词向量系数
                        input_length = MAX_SEQUENCE_LENGTH, # 每句话的最大长度（必须padding） 
                        trainable = False, # 是否在训练的过程中更新词向量
                       ),
             )

    ## input_shape=(Batch_size, Time_step, Input_Sizes)
    # 输入层
    model.add(SimpleRNN(128,
                        input_shape=(MAX_SEQUENCE_LENGTH, EMBEDDING_DIM),
                        activation='tanh', # 激活函数
                        return_sequences=True, # 返回全部time step的hidden state值
                       )
             )
    model.add(Dropout(0.5)) # 减少过拟合
    #model.add(Flatten())

    # 隐藏层
    model.add(Dense(64, 
                    input_shape=(MAX_SEQUENCE_LENGTH, 128), # 128维
                    activation='relu',
                   )
             )
    model.add(Dropout(0.5))

    # 输出层
    model.add(Dense(len(labels_types)+1,
                    input_shape=(MAX_SEQUENCE_LENGTH, 64),  # 64维
                    activation='softmax',
                   )
             )
    return model

model = RNN(embeddings_matrix, EMBEDDING_DIM, MAX_SEQUENCE_LENGTH, labels_types)

### （2）Bidirectional RNN

In [28]:
# BRNN模型
def BRNN(embeddings_matrix, EMBEDDING_DIM, MAX_SEQUENCE_LENGTH, labels_types):
    model = Sequential()
    model.add(Embedding(input_dim = len(embeddings_matrix), # 字典长度
                        output_dim = EMBEDDING_DIM, # 词向量长度（300）
                        weights = [embeddings_matrix], # 预训练的词向量系数
                        input_length = MAX_SEQUENCE_LENGTH, # 每句话的最大长度（必须padding） 
                        trainable = False, # 是否在训练的过程中更新词向量
                       ),
             )

    ## input_shape=(Batch_size, Time_step, Input_Sizes)
    # 输入层（添加bidirctional）
    model.add(Bidirectional(SimpleRNN(128, 
                                      input_shape=(MAX_SEQUENCE_LENGTH, EMBEDDING_DIM), 
                                      activation='tanh', 
                                      return_sequences=True
                                     ), 
                             merge_mode='concat',
                           ),
             )
    model.add(Dropout(0.5)) # 减少过拟合

    # 隐藏层
    model.add(Dense(64, 
                    input_shape=(MAX_SEQUENCE_LENGTH, 128), # 128维
                    activation='relu',
                   )
             )
    model.add(Dropout(0.5))

    # 输出层
    model.add(Dense(len(labels_types)+1,
                    input_shape=(MAX_SEQUENCE_LENGTH, 64),  # 64维
                    activation='softmax',
                   )
             )
    return model

model = BRNN(embeddings_matrix, EMBEDDING_DIM, MAX_SEQUENCE_LENGTH, labels_types)

In [29]:
## 未进行标签平滑
# 调节RMSprop的学习率参数lr为0.001
rms = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
model.compile(loss='categorical_crossentropy', # 损失函数
              optimizer=rms, # 优化器采用'rmsprop'
              metrics=['accuracy'], # 评价指标是准确率
             )

In [23]:
## 进行标签平滑
from keras.losses import CategoricalCrossentropy

# 标签平滑损失函数
loss = CategoricalCrossentropy(label_smoothing=0.1)

# 调节RMSprop的学习率参数lr为0.001
rms = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
model.compile(loss=loss, # 损失函数
              optimizer=rms, # 优化器采用'rmsprop'
              metrics=['accuracy'], # 评价指标是准确率
             )

In [30]:
# 输出模型各层的参数状况
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 300)          105665400 
_________________________________________________________________
bidirectional (Bidirectional (None, 100, 256)          109824    
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 256)          0         
_________________________________________________________________
dense_2 (Dense)              (None, 100, 64)           16448     
_________________________________________________________________
dropout_3 (Dropout)          (None, 100, 64)           0         
_________________________________________________________________
dense_3 (Dense)              (None, 100, 37)           2405      
Total params: 105,794,077
Trainable params: 128,677
Non-trainable params: 105,665,400
__________________________________

In [35]:
# 输出训练集中将词语转换为索引后的shape
print('训练集中将词语转换为索引后:', X_train_tokenized.shape)
# 18078个样本,每个样本100个词

# 输出训练集中将标签转换为索引并填充张量后的shape
print('\n训练集中将标签转换为索引并填充张量后', y_train_index_padded.shape)

训练集中将词语转换为索引后: (18078, 100)

训练集中将标签转换为索引并填充张量后 (18078, 100, 37)


In [31]:
# 使用训练集对模型进行训练
model.fit(X_train_tokenized,
          y_train_index_padded,
          epochs = 20,  # 训练20轮
          batch_size = 128, # 指定进行梯度下降时每个batch包含的样本数为128
          verbose = 1, # 展示训练过程
         )

# 评估训练完成的模型.返回损失值&模型的度量值.
print('-------------------Evaluation------------------------')
score = model.evaluate(X_test_tokenized, 
                       y_test_index_padded,
                       batch_size=128,
                      )

Epoch 1/20
142/142 [==============================] - 51s 357ms/step - loss: 0.5878 - accuracy: 0.8536
Epoch 2/20
142/142 [==============================] - 48s 337ms/step - loss: 0.2109 - accuracy: 0.9365
Epoch 3/20
142/142 [==============================] - 51s 361ms/step - loss: 0.1596 - accuracy: 0.9501
Epoch 4/20
142/142 [==============================] - 43s 302ms/step - loss: 0.1416 - accuracy: 0.9552
Epoch 5/20
142/142 [==============================] - 44s 307ms/step - loss: 0.1322 - accuracy: 0.9578
Epoch 6/20
142/142 [==============================] - 44s 309ms/step - loss: 0.1263 - accuracy: 0.9596
Epoch 7/20
142/142 [==============================] - 44s 312ms/step - loss: 0.1222 - accuracy: 0.9608
Epoch 8/20
142/142 [==============================] - 44s 312ms/step - loss: 0.1190 - accuracy: 0.9614
Epoch 9/20
142/142 [==============================] - 44s 307ms/step - loss: 0.1170 - accuracy: 0.9620
Epoch 10/20
142/142 [==============================] - 43s 306ms/step - l

In [ ]:
# 保存rnn模型
model.save('cn_pos_tag_rnn.h5')

In [ ]:
# 保存brnn模型
model.save('cn_pos_tag_brnn.h5')

In [32]:
print('训练的模型经在测试集上验证获得的loss和accuracy为：')
print(score)

训练的模型经在测试集上验证获得的loss和accuracy为：
[0.05298652499914169, 0.9812643527984619]


In [33]:
# 对测试集进行预测
print('-----------------------对测试集进行预测------------------------')
print('（1）对测试集 X_test_tokenized[:1] 进行预测：')
print(model.predict(X_test_tokenized[:1]))

-----------------------对测试集进行预测------------------------
（1）对测试集 X_test_tokenized[:1] 进行预测：
[[[2.9467149e-12 1.3376215e-18 6.6286404e-05 ... 3.3294791e-03
   1.7201482e-12 1.4566714e-13]
  [8.4083426e-19 7.2030519e-25 8.7497187e-07 ... 1.5803838e-08
   3.0710169e-23 8.6442037e-18]
  [1.3564103e-21 1.0349561e-24 6.6359036e-08 ... 3.0411465e-12
   1.7746174e-18 3.3569497e-17]
  ...
  [9.9999988e-01 2.0126665e-27 8.7387924e-08 ... 3.3107492e-14
   1.3039966e-23 6.8032123e-18]
  [9.9999964e-01 5.3980682e-25 3.1134417e-07 ... 6.8773719e-13
   1.6942969e-21 2.3384099e-16]
  [9.9999857e-01 1.4131538e-22 1.1144496e-06 ... 1.7796905e-11
   3.4814879e-19 1.2530364e-14]]]


In [34]:
# 输出测试集中标签填充的张量
print('测试集中标签填充的张量 y_test_index_padded[:1] 为：')
print(y_test_index_padded[:1])

测试集中标签填充的张量 y_test_index_padded[:1] 为：
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]]


In [35]:
# 对测试集进行预测
print('（2）对测试集 X_test_tokenized[2: 3] 进行预测：')
print(model.predict(X_test_tokenized[2: 3]))

（2）对测试集 X_test_tokenized[2: 3] 进行预测：
[[[2.1755171e-14 5.7821930e-28 9.9999845e-01 ... 1.0647330e-14
   5.0780611e-23 1.7765798e-08]
  [1.5606285e-11 7.1746576e-16 8.5157044e-05 ... 5.7841246e-03
   3.4609267e-11 4.5957384e-13]
  [2.0214038e-18 1.9453082e-25 2.9080093e-06 ... 2.8483396e-09
   1.4453245e-21 1.1426978e-18]
  ...
  [9.9999988e-01 2.0126665e-27 8.7387924e-08 ... 3.3107428e-14
   1.3039966e-23 6.8032123e-18]
  [9.9999964e-01 5.3981091e-25 3.1134476e-07 ... 6.8773719e-13
   1.6943034e-21 2.3384099e-16]
  [9.9999857e-01 1.4131538e-22 1.1144484e-06 ... 1.7796905e-11
   3.4815014e-19 1.2530317e-14]]]


In [60]:
# 输出测试集中标签填充的张量
print('测试集中标签填充的张量 y_test_index_padded[:1] 为：')
print(y_test_index_padded[:1])

测试集中标签填充的张量 y_test_index_padded[:1] 为：
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
